# This notebook demonstrates drift detection for text data with evidently using IMDB reviews and eco-dot reviews.

#### In the first part of the notebook we demonstrate using glove vectors and in the later part of the notebook we show how sentence transformers could be utilized.

#### We demonstrate the usage with both glove vectors and sentence transformers

#### In case of glove vectors, each record is represented as a sentence vector by using glove6B 50d vector (by averaging) for both IMDB and eco-dot reviews datasets. Incase a vector representation is not present for a particular word, we use 50d zero vector.

#### In case of sentence transformers we use the MiniLM-v2 model to get the embeddings which is a 384 vector representation. This is 

# Install and import the pre-requisites

In [ ]:
try:
    import evidently
except:
    !pip install git+https://github.com/evidentlyai/evidently.git

In [ ]:
# Install sentence transformers
!pip install sentence-transformers

In [ ]:
import pandas as pd
import numpy as np

from sklearn import datasets

from evidently import ColumnMapping
from evidently.report import Report
from evidently.metrics import EmbeddingsDriftMetric
from sklearn.model_selection import train_test_split
from evidently.metrics.data_drift.embedding_drift_methods import model, distance, ratio, mmd

# Load cleaned Imdb movies review dataset and amazon ecodot review dataset

The datasets used in this notebook are from the below resource

https://github.com/SangamSwadiK/nlp_example_datasets

In case, you do not want to load the glove vectors, and want to use preprocessed vectors and view the drift, please go to https://github.com/SangamSwadiK/nlp_example_datasets and use the rawgithub url's or download it and load them.

#### Please go through the below kaggle resources for pre-processing and if you want to acess the original datasets:
1) https://www.kaggle.com/code/lakshmi25npathi/sentiment-analysis-of-imdb-movie-reviews

2) https://www.kaggle.com/datasets/PromptCloudHQ/amazon-echo-dot-2-reviews-dataset

In [ ]:
# IMBD reviews dataset
imdb_5k_data = pd.read_csv("https://raw.githubusercontent.com/SangamSwadiK/test_dataset/main/cleaned_imdb_data_5000_rows.csv")
imdb_5k_data.head()

In [ ]:
#This data is oob and has no relationship with movie reviews, you can replace the test set with eco_dot to better understand the workings of drift detection.
eco_dot_data = pd.read_csv("https://raw.githubusercontent.com/SangamSwadiK/test_dataset/main/eco_data.csv", squeeze=True)
eco_dot_data.head()

In [ ]:
## Run this to experiment with the dataset with various ways of embedding (average over records / sum of records etc ...)
!wget http://nlp.stanford.edu/data/glove.6B.zip -P /content/test/
!unzip  /content/test/glove.6B.zip -d /content/test/

In [ ]:
# Load glove vector from vector file
def load_glove_model(File):
  """ Loads the keyed vectors from a given text file
  Args:
    File: text file which contains the vectors
  Returns:
    Dictionary: map containing the key:vector pair
  """
  glove_model = {}
  with open(File,'r') as f:
      for line in f:
          split_line = line.split()
          word = split_line[0]
          embedding = np.array(split_line[1:], dtype=np.float64)
          glove_model[word] = embedding
      return glove_model

In [ ]:
# We load 50 dimension vector here
glove_vec = load_glove_model("/content/test/glove.6B.50d.txt")

In [8]:
# We load 50 dimension vector here
glove_vec = load_glove_model("/content/test/glove.6B.50d.txt")

# Train test split the imdb dataset for input data drift comparison

In [9]:
## Perform train test split on imdb data
train_df, test_df, y_train, y_test = train_test_split(imdb_5k_data.review, imdb_5k_data.sentiment, test_size=0.50, random_state=42)

# Convert train and test records into embedding vectors

In [10]:
def get_sentence_vector(dataframe):
  """Get a sentence vector for each text/record by averaging based on counts for each text record
  Args:
    dataframe: the dataframe containing the text data
  returns:
    array: a matrix of sentence vectors for each record in the dataframe
  """
  tmp_arr = []
  for row in dataframe.values:
    tmp = np.zeros(50,)
    for word in row:
      try:
        tmp += glove_vec[word]
      except KeyError:
        tmp+= np.zeros(50,)
    tmp = tmp/len(row.split(" "))
    tmp_arr.append(tmp.tolist())

  return tmp_arr

In [11]:
train_matrix =  get_sentence_vector(train_df)

In [12]:
train_df_converted = pd.DataFrame(np.array(train_matrix), index = train_df.index)
train_df_converted.columns = ["col_"+ str(i) for i in range(train_df_converted.shape[1])]
train_df_converted.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,...,col_40,col_41,col_42,col_43,col_44,col_45,col_46,col_47,col_48,col_49
1511,-0.381491,3.842186,3.163375,3.268990,1.716181,-0.244856,1.811342,-3.424588,-1.184743,-0.061534,...,3.774490,0.275538,-0.955406,-0.215360,-0.306830,1.241856,0.867399,-1.963373,1.666094,4.384211
1882,-0.364928,3.428283,2.872637,2.932077,1.422103,-0.258355,1.755376,-3.027312,-1.097868,-0.105749,...,3.441375,0.266090,-0.893476,-0.166002,-0.255953,1.197352,0.726796,-1.875431,1.364674,4.111564
651,-0.225922,3.184136,2.519968,2.971848,1.068490,-0.218869,1.504055,-2.913288,-0.902471,0.042850,...,3.329671,0.254596,-0.829084,-0.329340,-0.153338,0.862841,0.607194,-1.684554,1.477441,3.787845
4930,-0.261043,3.496850,2.649218,2.948960,1.283928,-0.351531,1.672046,-2.960629,-1.044285,-0.051660,...,3.454432,0.194855,-0.792499,-0.328251,-0.305474,1.009205,0.686892,-1.852934,1.563900,3.740972
3089,-0.371894,3.544178,3.041681,2.999930,1.551454,-0.209505,1.713554,-3.216542,-1.076544,-0.207425,...,3.562595,0.262266,-0.921348,-0.065731,-0.299506,1.192069,0.847945,-1.786974,1.450985,4.078656


In [13]:
## Get the sentence vectors for test dataframe
test_matrix =  get_sentence_vector(test_df)

In [14]:
test_df_converted = pd.DataFrame(np.array(test_matrix), index = test_df.index)
test_df_converted.columns = ["col_"+ str(i) for i in range(test_df_converted.shape[1])]

test_df_converted.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,...,col_40,col_41,col_42,col_43,col_44,col_45,col_46,col_47,col_48,col_49
1501,-0.301892,3.456919,2.805822,2.952740,1.545965,-0.364327,1.792683,-2.878874,-1.229111,0.017821,...,3.368028,0.215296,-0.854739,-0.269794,-0.354434,1.066241,0.728645,-1.925135,1.520077,4.230904
2586,-0.427536,3.647496,2.882300,3.151114,1.414213,-0.349614,1.707540,-3.145892,-1.009917,-0.073269,...,3.537440,0.263921,-0.778365,-0.232846,-0.286558,1.135535,0.738475,-1.965921,1.504252,4.090509
2653,-0.322373,3.220931,2.661710,2.703467,1.460842,-0.336064,1.524281,-2.796387,-0.977345,-0.039393,...,3.239543,0.177134,-0.768166,-0.133148,-0.171684,1.097073,0.691741,-1.682584,1.445865,3.591712
1055,-0.234422,3.295430,2.769252,2.916453,1.346506,-0.195492,1.554906,-2.946577,-1.006702,-0.067785,...,3.485205,0.271937,-0.748010,-0.253616,-0.213026,1.186579,0.736563,-1.736270,1.447214,3.903953
705,-0.298585,3.467689,2.833680,3.010032,1.445126,-0.295791,1.642785,-3.097794,-1.077196,0.023100,...,3.483806,0.254906,-0.836744,-0.211576,-0.345574,1.087492,0.762931,-1.854362,1.533544,4.021160


In [15]:
# Get sentence vector for echo dot
eco_dot_matrix = get_sentence_vector(eco_dot_data)

In [16]:
ecodot_review_df_converted = pd.DataFrame(np.array(eco_dot_matrix), index = eco_dot_data.index)
ecodot_review_df_converted.columns = ["col_"+ str(i) for i in range(ecodot_review_df_converted.shape[1])]

ecodot_review_df_converted.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,...,col_40,col_41,col_42,col_43,col_44,col_45,col_46,col_47,col_48,col_49
0,0.327875,4.016490,2.791180,4.144132,1.833375,-0.312672,1.287728,-4.142395,-1.229878,1.161340,...,3.918637,-0.035790,-1.074380,-0.550616,-0.885010,0.599091,0.867318,-1.611072,3.009166,3.895240
1,-0.353587,4.744348,4.072236,4.225321,1.609850,-0.012264,2.406608,-4.641343,-1.675868,-0.039295,...,4.847699,0.667103,-1.344172,-0.314820,-0.918985,2.066251,1.230749,-2.030697,2.029354,5.928920
2,-0.436141,5.019880,3.867931,4.208932,1.866874,-0.683997,2.530752,-4.288440,-1.314428,0.170220,...,4.881951,0.363756,-0.983098,-0.482030,-0.655690,1.638361,1.098308,-2.558459,2.373431,4.871069
3,-0.520974,5.434466,3.984750,4.761608,1.522328,-0.786353,2.435981,-5.003207,-1.356486,0.155528,...,5.576414,0.392405,-0.879580,-0.363400,-0.971590,1.417375,1.163867,-2.981916,2.536001,5.487957
4,-0.445168,5.124225,3.961621,5.011146,1.692521,-0.773384,2.372935,-4.812509,-1.781645,0.553531,...,5.247725,0.419570,-1.119463,-0.591218,-0.662437,1.414253,0.961471,-2.823617,2.595306,5.771914


# Embeddings Drift Report

Here we take a small subset of the columns and calculate the drift based on it.
To understand more about how its being calculated checkout the below link

https://github.com/evidentlyai/evidently/blob/476a14152799df0c0f0701014b8717d43585fb6b/src/evidently/metrics/data_drift/embeddings_drift.py#L27

In [17]:
column_mapping = ColumnMapping(
    embeddings={'small_subset': train_df_converted.columns[:10]}
)

In [36]:
# Here we measure drift on the small subset between train and test imdb records
report = Report(metrics=[
    EmbeddingsDriftMetric('small_subset')
])

report.run(reference_data = train_df_converted[:500], current_data = test_df_converted[500:1000], 
           column_mapping = column_mapping)
report

# Embeddings Drift Detection: model

This approach involves training an SGD Classifier and calculating the ROC AUC score. The drift is measured on the same. If bootstrap and PCA components are enabled, it performs dimensionality reduction with PCA and then performs bootstrap for the ROC AUC and returns the drift result

Checkout the below link to understand more on how its calculated

https://github.com/evidentlyai/evidently/blob/476a14152799df0c0f0701014b8717d43585fb6b/src/evidently/metrics/data_drift/embedding_drift_methods.py#L99

In [37]:
report = Report(metrics = [
    EmbeddingsDriftMetric('small_subset', 
                          drift_method = model(
                              threshold = 0.55,
                              bootstrap = None,
                              quantile_probability = 0.95,
                              pca_components = None,
                          )
                         )
])

report.run(reference_data = train_df_converted[:500], current_data = test_df_converted[500:1000],  
           column_mapping = column_mapping)
report

# Embeddings Drift Detection: mmd

Here, The drift is calculated using the maximum mean discrepancy test (MMD).
If you want to learn more about MMD checkout the below paper
https://www.jmlr.org/papers/volume13/gretton12a/gretton12a.pdf

If you want to understand how MMD is used as a Multivariate test checkout the below paper

https://arxiv.org/pdf/1810.11953.pdf


To understand more about the implementation checkout the below

https://github.com/evidentlyai/evidently/blob/476a14152799df0c0f0701014b8717d43585fb6b/src/evidently/metrics/data_drift/embedding_drift_methods.py#L201

In [38]:
report = Report(metrics = [
    EmbeddingsDriftMetric('small_subset', 
                          drift_method = mmd(
                              threshold = 0.015,
                              bootstrap = None,
                              quantile_probability = 0.95,
                              pca_components = None,
                          )
                         )
])

report.run(reference_data = train_df_converted[:500], current_data = test_df_converted[500:1000],  
           column_mapping = column_mapping)
report

# Embeddings Drift Detection: ratio

Here, The drift is calculated based on the ratio of drifted embeddings, we look at each individual embedding and then apply a statistical test which can be picked from the evidently stattests module (https://docs.evidentlyai.com/reference/api-reference/evidently.calculations/evidently.calculations.stattests)

In case you want to reduce the dimensionality and use this method, use the pca_components parameter.

Checkout the below link to understand more about how this works

https://github.com/evidentlyai/evidently/blob/476a14152799df0c0f0701014b8717d43585fb6b/src/evidently/metrics/data_drift/embedding_drift_methods.py#L139

In [39]:
report = Report(metrics = [
    EmbeddingsDriftMetric('small_subset', 
                          drift_method = ratio(
                              component_stattest = 'wasserstein',
                              component_stattest_threshold = 0.1,
                              threshold = 0.2,
                              pca_components = None,
                          )
                         )
])

report.run(reference_data = train_df_converted[:500], current_data = test_df_converted[500:1000],  
           column_mapping = column_mapping)
report

# Embeddings Drift detection: Distance

Here we use the average distance method for measuring the drift 
The available distances are euclidean, cosine, cityblock and chebyshev.

If bootstrap is enabled, it performs bootstrapping to calculate drift based on quantile probability. If not enabled, it uses the threshold parameter. All values above this threshold means data drift. This only applies when bootstrap != True


To understand how this is implemented checkout the below link

https://github.com/evidentlyai/evidently/blob/476a14152799df0c0f0701014b8717d43585fb6b/src/evidently/metrics/data_drift/embedding_drift_methods.py#L45

In [40]:
report = Report(metrics = [
    EmbeddingsDriftMetric('small_subset', 
                          drift_method = distance(
                              dist = 'euclidean', #"euclidean", "cosine", "cityblock" or "chebyshev"
                              threshold = 0.2,
                              pca_components = None,
                              bootstrap = None,
                              quantile_probability = 0.95
                          )
                         )
])

report.run(reference_data = train_df_converted[:500], current_data = test_df_converted[500:1000],  
           column_mapping = column_mapping)
report

# Drift detection with sentence transformers

This demonstrates usage of sentence transformers with evidently for drift detection

## Convert to embeddings

In [23]:
# import MiniLM v2 from sentence transformer

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

In [24]:
# Encode only a fraction
ref_embeddings = model.encode(imdb_5k_data["review"][: 100].tolist() )

In [25]:
ref_df = pd.DataFrame(ref_embeddings)
ref_df.columns = ['col_' + str(x) for x in ref_df.columns]
ref_df.head(5)

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,...,col_374,col_375,col_376,col_377,col_378,col_379,col_380,col_381,col_382,col_383
0,0.021698,0.018661,-0.066963,0.050993,0.069774,0.066455,0.040649,0.043083,0.053272,-0.054460,...,0.045743,-0.071286,-0.018814,0.033319,-0.006450,-0.009169,0.052393,-0.013439,-0.006549,-0.023041
1,-0.041562,0.030806,-0.029254,-0.057882,0.020065,0.095900,-0.019118,-0.014568,-0.029918,-0.044381,...,0.009234,0.045543,0.022432,0.033648,-0.070079,0.008358,0.047206,0.001128,-0.051015,-0.012015
2,-0.016844,-0.060643,-0.009080,-0.016619,0.070509,0.072632,0.086838,0.087207,-0.027978,-0.004109,...,-0.019848,-0.076331,-0.123335,0.059031,-0.078645,0.027605,0.046011,-0.007313,-0.003385,-0.077472
3,-0.068421,0.014181,-0.062271,-0.025391,0.059005,0.090587,0.033989,-0.021858,0.112777,-0.059827,...,0.051906,-0.025077,0.013942,-0.007221,-0.010040,0.022526,0.095781,0.024663,0.034347,0.017130
4,-0.086811,-0.074347,-0.068667,-0.058436,-0.048506,0.064504,0.101687,0.046137,0.090395,-0.042411,...,-0.043221,-0.075269,-0.024872,0.129075,0.005491,-0.039255,0.025810,-0.037690,0.037844,-0.006613


In [26]:
# Similarly encode only a fraction
cur_embeddings = model.encode( eco_dot_data.tolist()[:100] )

In [27]:
cur_df = pd.DataFrame(cur_embeddings)
cur_df.columns = ['col_' + str(x) for x in cur_df.columns]
cur_df.head(5)

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,...,col_374,col_375,col_376,col_377,col_378,col_379,col_380,col_381,col_382,col_383
0,0.035804,-0.144474,0.033393,-0.093649,-0.012871,0.034773,-0.070291,-0.066118,-0.026981,0.020925,...,0.048090,0.041775,0.072481,-0.042344,-0.079178,0.041376,0.037252,0.075955,0.125962,-0.018965
1,-0.034748,-0.093350,0.048463,-0.058598,-0.045063,0.061187,0.044661,-0.064357,-0.021010,-0.030826,...,0.064003,0.028441,0.037830,0.017256,0.011066,0.082818,0.020051,0.085044,0.006845,0.005290
2,-0.062197,-0.071352,-0.052099,-0.047364,-0.034019,-0.012032,-0.022654,-0.027985,0.021279,0.017623,...,0.083788,-0.032237,-0.009527,0.039137,0.008174,0.099074,0.045092,-0.005761,-0.008619,0.035169
3,-0.084088,-0.104216,-0.036165,-0.094645,-0.020838,0.040887,0.021206,-0.068479,-0.060041,-0.088374,...,0.123519,0.032993,0.012024,0.009120,0.056981,0.132225,0.004404,0.022267,-0.003054,0.003895
4,-0.070356,0.001550,-0.012380,0.025694,-0.016914,-0.009343,0.006640,0.021307,-0.012566,0.023944,...,0.029027,0.046931,-0.088979,0.073582,-0.021607,0.026900,0.045848,-0.173566,0.007128,0.046765


## Embeddings Drift Report


Here we take a small subset of the columns and calculate the drift based on it.
To understand more about how its being calculated checkout the below link

https://github.com/evidentlyai/evidently/blob/476a14152799df0c0f0701014b8717d43585fb6b/src/evidently/metrics/data_drift/embeddings_drift.py#L27

In [28]:
column_mapping = ColumnMapping(
    embeddings={'small_subset': ref_df.columns[:10]}
)

In [41]:
report = Report(metrics=[
    EmbeddingsDriftMetric('small_subset')
])

report.run(reference_data = ref_df[:50], current_data = cur_df[:50], 
           column_mapping = column_mapping)
report

## Embeddings Drift Detection: model

This approach involves training an SGD Classifier and calculating the ROC AUC score. The drift is measured on the same. If bootstrap and PCA components are enabled, it performs dimensionality reduction with PCA and then performs bootstrap for the ROC AUC and returns the drift result

Checkout the below link to understand more on how its calculated

https://github.com/evidentlyai/evidently/blob/476a14152799df0c0f0701014b8717d43585fb6b/src/evidently/metrics/data_drift/embedding_drift_methods.py#L99

In [42]:
report = Report(metrics = [
    EmbeddingsDriftMetric('small_subset', 
                          drift_method = model(
                              threshold = 0.55,
                              bootstrap = None,
                              quantile_probability = 0.95,
                              pca_components = None,
                          )
                         )
])

report.run(reference_data = ref_df[:50], current_data = cur_df[:50], 
           column_mapping = column_mapping)
report

## Embeddings Drift Detection: MMD

Here, The drift is calculated using the maximum mean discrepancy test (MMD). If you want to learn more about MMD checkout the below paper https://www.jmlr.org/papers/volume13/gretton12a/gretton12a.pdf

If you want to understand how MMD is used as a Multivariate test checkout the below paper

https://arxiv.org/pdf/1810.11953.pdf

To understand more about the implementation checkout the below

https://github.com/evidentlyai/evidently/blob/476a14152799df0c0f0701014b8717d43585fb6b/src/evidently/metrics/data_drift/embedding_drift_methods.py#L201

In [43]:
report = Report(metrics = [
    EmbeddingsDriftMetric('small_subset', 
                          drift_method = mmd(
                              threshold = 0.015,
                              bootstrap = None,
                              quantile_probability = 0.95,
                              pca_components = None,
                          )
                         )
])

report.run(reference_data = ref_df[:50], current_data = ref_df[:50],  
           column_mapping = column_mapping)
report

## Embeddings Drift Detection: ratio

Here, The drift is calculated based on the ratio of drifted embeddings, we look at each individual embedding and then apply a statistical test which can be picked from the evidently stattests module (https://docs.evidentlyai.com/reference/api-reference/evidently.calculations/evidently.calculations.stattests)

In case you want to reduce the dimensionality and use this method, use the pca_components parameter.

Checkout the below link to understand more about how this works

https://github.com/evidentlyai/evidently/blob/476a14152799df0c0f0701014b8717d43585fb6b/src/evidently/metrics/data_drift/embedding_drift_methods.py#L139

In [44]:
report = Report(metrics = [
    EmbeddingsDriftMetric('small_subset', 
                          drift_method = ratio(
                              component_stattest = 'wasserstein',
                              component_stattest_threshold = 0.1,
                              threshold = 0.2,
                              pca_components = None,
                          )
                         )
])

report.run(reference_data = ref_df[:50], current_data = ref_df[:50],  
           column_mapping = column_mapping)
report

## Embeddings Drift detection: Distance

Here we use the average distance method for measuring the drift 
The available distances are euclidean, cosine, cityblock and chebyshev.

If bootstrap is enabled, it performs bootstrapping to calculate drift based on quantile probability. If not enabled, it uses the threshold parameter. All values above this threshold means data drift. This only applies when bootstrap != True


To understand how this is implemented checkout the below link

https://github.com/evidentlyai/evidently/blob/476a14152799df0c0f0701014b8717d43585fb6b/src/evidently/metrics/data_drift/embedding_drift_methods.py#L45

In [45]:
report = Report(metrics = [
    EmbeddingsDriftMetric('small_subset', 
                          drift_method = distance(
                              dist = 'euclidean', #"euclidean", "cosine", "cityblock" or "chebyshev"
                              threshold = 0.2,
                              pca_components = None,
                              bootstrap = None,
                              quantile_probability = 0.95
                          )
                         )
])

report.run(reference_data = ref_df[:50], current_data = ref_df[:50],  
           column_mapping = column_mapping)
report